In [ ]:
#| default_exp datasources.aisgeo
%load_ext autoreload
%autoreload 2

import sys, os
from pathlib import Path

# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))
os.chdir(Path.cwd().parent / 'extracao')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# GEOAISWEB

> Este módulo concentra as constantes, funções de carga, processamento, mesclagem e salvamento de dados aeronáuticos provenientes da API do GeoAisWeb

In [ ]:
#| export
import json
import os
from pathlib import Path
from typing import List
from urllib.request import urlopen

import pandas as pd
from dotenv import find_dotenv, load_dotenv
from extracao.constants import VOR_ILS_DME

load_dotenv(find_dotenv(), override=True)

True

## CONSTANTES


Dados para acesso à API GEOAISWEB

In [ ]:
#| export
LINK_VOR = 'https://geoaisweb.decea.mil.br/geoserver/ICA/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=ICA:vor&outputFormat=application%2Fjson'
LINK_DME = 'https://geoaisweb.decea.mil.br/geoserver/ICA/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=ICA:dme&outputFormat=application%2Fjson'
LINK_NDB = 'https://geoaisweb.decea.mil.br/geoserver/ICA/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=ICA:ndb&outputFormat=application%2Fjson'
COLS_VOR = (
	'properties.frequency',
	'properties.frequnits',
	'properties.latitude',
	'properties.longitude',
	'properties.tipo',
	'properties.txtname',
	'properties.txtrmk',
)
COLS_NDB = (
	'properties.valfreq',
	'properties.uomfreq',
	'properties.geolat',
	'properties.geolong',
	'properties.tipo',
	'properties.txtname',
	'properties.txtrmk',
)

COLS_DME = (
	'properties.valchannel',
	'properties.codechanne',
	'properties.geolat',
	'properties.geolong',
	'properties.tipo',
	'properties.txtname',
	'Channel',
)

UNIQUE_COLS = ['Frequência', 'Latitude', 'Longitude']


In [ ]:
#| export
def convert_frequency(
	freq: float,  # Frequência Central da emissão
	unit: str,  # Unidade da Frequência: [Hz, kHz, MHZ, GHZ]
) -> float:  # Frequência em MHz
	"""Converte a frequência `freq` para MHz"""
	unit = unit.upper()
	if unit == 'GHZ':
		return freq * 1000
	elif unit == 'KHZ':
		return freq / 1000
	elif unit == 'HZ':
		return freq / 1e6
	elif unit == 'MHZ':
		return freq
	else:
		return -1


In [ ]:
#| export
def _process_frequency(
	df: pd.DataFrame,  # Dataframe com os dados
	cols: List[str],  # Subconjunto de Colunas relevantes do DataFrame
) -> pd.DataFrame:  # Dataframe com os dados de frequência devidamente processados
	if cols == COLS_DME:
		df_channels = pd.read_csv(VOR_ILS_DME, dtype='string', dtype_backend='pyarrow')
		df = df.dropna(subset=[cols[0]])
		df['Channel'] = df[cols[0]].astype('int').astype('string') + df[cols[1]]
		df['Frequência'] = -1.0

		for row in df.itertuples(index=True):
			row_match = df_channels.loc[(df_channels.Channel == row.Channel), 'DMEground']
			if not row_match.empty:
				df.loc[row.Index, 'Frequência'] = float(row_match.item())

	else:
		df['Frequência'] = (
			df[[cols[0], cols[1]]]
			.apply(lambda x: convert_frequency(x[0], x[1]), axis=1)
			.astype('float')
		)
	return df


In [ ]:
#| export
def _filter_df(df, cols):  # sourcery skip: use-fstring-for-concatenation
	df.fillna('', inplace=True)
	df['Entidade'] = (df[cols[4]] + ' - ' + df[cols[5]] + ' ' + df[cols[6]]).astype('string')
	df['Fonte'] = 'AISGEO'
	df = df[['Frequência', cols[2], cols[3], 'Entidade', 'Fonte']]
	return df.rename(
		columns={
			cols[2]: 'Latitude',
			cols[3]: 'Longitude',
		}
	)


In [ ]:
#|export
def get_geodf(
	link: str,  # Link para a requisição das estações VOR do GEOAISWEB
	cols: List[str],  # Subconjunto de Colunas relevantes do DataFrame
) -> pd.DataFrame:  # DataFrame com frequências, coordenadas e descrição das estações VOR
	# sourcery skip: use-fstring-for-concatenation
	"""Faz a requisição do `link`, processa o json e o retorna como Dataframe"""
	response = urlopen(link)
	if response.status != 200 or 'application/json' not in response.headers['content-type']:
		raise ValueError(f'Resposta a requisição não foi bem sucedida: {response.status=}')
	data_json = json.loads(response.read())
	df = pd.json_normalize(
		data_json['features'],
	).filter(cols, axis=1)
	df = _process_frequency(df, cols)
	return _filter_df(df, cols)


In [ ]:
#| eval: false
get_geodf(LINK_VOR, COLS_VOR)

Frequency   Latitude  Longitude  \
0       116.8 -16.245367 -48.979089   
1       117.0 -19.689048 -47.060575   
2       113.4  -9.868361 -56.104951   
3       116.2 -22.951451 -46.569805   
4       114.3 -12.079940 -45.007135   
..        ...        ...        ...   
74      112.1 -25.583203 -54.503514   
75      115.3 -31.390714 -54.109717   
76      116.9 -23.627464 -46.654635   
77      115.9 -14.799000 -64.938333   
78      115.9 -25.244167 -57.522000   

                                          Description  
0                       [AISG] VOR - ANÁPOLIS CH 101X  
1                                 [AISG] VOR - ARAXÁ   
2      [AISG] VOR - ALTA FLORESTA CH 81X
OPR INFRAERO  
3                       [AISG] VOR - BRAGANÇA CH 109X  
4                       [AISG] VOR - BARREIRAS CH 90X  
..                                                ...  
74                            [AISG] VOR - FOZ CH 58X  
75              [AISG] VOR - BAGÉ U/S BTN RDL 275/305  
76  [AISG] VOR - CONGONHAS CH 116X \nVOR/DME NO AV...  
77           [AISG] VOR - TRINIDAD-BL See Bolivia AIP  
78       [AISG] VOR - ASUNCION VDVAS See Paraguay AIP  

[79 rows x 4 columns]

In [ ]:
#| eval: false
get_geodf(LINK_NDB, COLS_NDB)

Frequency   Latitude  Longitude  \
0      0.2650 -21.139333 -50.426667   
1      0.3000 -25.402667 -49.229000   
2      0.3800 -22.314000 -49.107167   
3      0.2300  -7.266000 -35.892667   
4      0.4200 -20.813167 -49.406167   
5      0.4070  -4.194997 -69.939733   
6      0.2500 -29.694722 -57.148056   
7      0.2950 -19.658900 -43.896933   
8      0.3750 -19.016219 -57.664456   
9      0.2050  -5.386167 -35.531000   
10     0.4100  -3.142825 -59.993747   
11     0.3000 -17.217111 -46.933821   
12     0.2050  -1.486333 -56.397833   
13     0.4300 -23.045636 -45.516708   
14     0.3550  -4.886500 -65.349833   
15     0.2850 -22.197333 -54.928333   
16     0.2750 -22.787975 -45.215286   
17     0.3400 -22.168889 -49.069722   
18     0.1143 -32.340056 -54.223889   
19     0.2800 -28.678850 -49.066714   
20     0.2850 -21.267053 -43.764111   
21     0.3400   3.860144 -51.800131   
22     0.2900 -17.724656 -48.610410   
23     0.4000 -10.784333 -65.281167   
24     0.2000 -22.923333 -42.071500   
25     0.2400   2.219028 -55.941800   
26     0.4150 -22.194833 -49.929333   
27     0.3200  -7.140500 -34.952500   
28     0.3850 -16.441667 -39.082500   
29     0.2300  -4.249333 -69.936167   
30     0.2750 -29.786014 -57.036586   
31     0.2800 -28.279311 -54.171944   
32     0.2250 -25.408667 -54.621667   
33     0.3400 -22.550075 -55.707161   
34     0.2950  -3.256500 -52.250833   
35     0.4100 -23.462189 -46.573404   
36     0.3100 -16.703383 -43.819108   
37     0.4150 -22.870108 -43.165119   
38     0.3300 -21.142667 -47.776000   
39     0.2350 -19.765500 -47.959167   
40     0.2500  -4.240500 -56.004667   
41     1.6180 -19.463000 -42.479500   
42     0.3000 -29.996308 -50.145006   
43     0.3300 -16.335333 -58.387167   

                                          Description  
0                      [AISG] NDB - ARAÇATUBA OPR TAM  
1        [AISG] NDB - BACACHERI FM ARP, COVERAGE 50NM  
2       [AISG] NDB - BAURU COVERAGE 50NM
OPR INFRAERO  
3   [AISG] NDB - CAMPINA GRANDE COVERAGE 60NM
OPR ...  
4                    [AISG] NDB - RIO PRETO OPR DAESP  
5                   [AISG] NDB - LETÍCIA OPR COLÔMBIA  
6       [AISG] NDB - PASO DE LOS LIBRES OPR ARGENTINA  
7              [AISG] NDB - LAGOA SANTA COVERAGE 50NM  
8                               [AISG] NDB - CORUMBÁ   
9                          [AISG] NDB - MAXARANGUAPE   
10                  [AISG] NDB - PELADA COVERAGE 70NM  
11                [AISG] NDB - PARACATU COVERAGE 65NM  
12  [AISG] NDB - TROMBETAS COVERAGE 100NM; OPR MIN...  
13  [AISG] NDB - TAUBATÉ COVERAGE 25NM; OPR COMAND...  
14                   [AISG] NDB - URUCU OPR PETROBRAS  
15                   [AISG] NDB - DOURA COVERAGE 70NM  
16           [AISG] NDB - GUARATINGUETÁ COVERAGE 50NM  
17                              [AISG] NDB - AREALVA   
18                                 [AISG] NDB - MELO   
19  [AISG] NDB - JAGUARUNA COVERAGE 30NM. Other HR...  
20               [AISG] NDB - BARBACENA COVERAGE 60NM  
21                [AISG] NDB - OIAPOQUE COVERAGE 70NM  
22  [AISG] NDB - CALDAS NOVAS OPR CALDAS NOVAS CIT...  
23                        [AISG] NDB - GUAJARÁ-MIRIM   
24  [AISG] NDB - CABO FRIO OPR COSTA DO SOL OPERAD...  
25                 [AISG] NDB - TIRIÓS COVERANGE 50NM  
26   [AISG] NDB - MARÍLIA OPR PANTANAL LINHAS AÉREAS.  
27  [AISG] NDB - JOAO PESSOA Coverage 40NM
OPR INF...  
28           [AISG] NDB - PORTO SEGURO COVERAGE 27 NM  
29               [AISG] NDB - TABATINGA COVERAGE 50NM  
30             [AISG] NDB - URUGUAIANA COVERAGE 100NM  
31             [AISG] NDB - SANTO ÂNGELO OPR RIO SUL.  
32                               [AISG] NDB - ITAIPU   
33               [AISG] NDB - PONTA PORÃ OPR INFRAERO  
34               [AISG] NDB - ALTAMIRA OPR NAV Brasil  
35                   [AISG] NDB - TUCA OPR NAV Brasil  
36  [AISG] NDB - MONTES CLAROS COVERAGE 50NM
OPR N...  
37                  [AISG] NDB - PAIOL OPR NAV Brasil  
38  [AISG] NDB - RIBEIRÃO COVERAGE 50NM
OPR NAV Br...  
39  [AIS

In [ ]:
#| eval: false
get_geodf(LINK_DME, COLS_DME)

C:\Users\rsilva\AppData\Local\Temp\2\ipykernel_7420\3107854738.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Channel"] = df[cols[0]].astype("int").astype("string") + df[cols[1]]
C:\Users\rsilva\AppData\Local\Temp\2\ipykernel_7420\3107854738.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Frequency"] = -1.0


Frequency   Latitude  Longitude                            Description
0       1019.0 -22.812774 -42.095339                [AISG] DME - ALDEIA 58X
1       1202.0 -16.245367 -48.979089             [AISG] DME - ANÁPOLIS 115X
2       1204.0 -19.689048 -47.060575                [AISG] DME - ARAXÁ 117X
3       1168.0  -9.868361 -56.104951         [AISG] DME - ALTA FLORESTA 81X
4       1196.0 -22.951358 -46.569900             [AISG] DME - BRAGANÇA 109X
..         ...        ...        ...                                    ...
148     1175.0 -25.537761 -48.529855             [AISG] DME - PARANAGUÁ 88X
149     1181.0 -18.203235 -45.457072           [AISG] DME - TRÊS MARIAS 94X
150     1171.0 -27.621855 -48.632464               [AISG] DME - BIGUAÇU 84X
151     1186.0 -14.907781 -40.918839  [AISG] DME - VITÓRIA DA CONQUISTA 99X
152     1176.0 -19.442511 -44.231072           [AISG] DME - SETE LAGOAS 89X

[150 rows x 4 columns]

In [ ]:
#| export
def get_aisg() -> pd.DataFrame:  # DataFrame com todos os dados do GEOAISWEB
	"""Lê e processa os dataframes individuais da API GEOAISWEB e retorna o conjunto concatenado"""
	df = pd.concat(
		get_geodf(link, cols)
		for link, cols in zip([LINK_NDB, LINK_VOR, LINK_DME], [COLS_NDB, COLS_VOR, COLS_DME])
	)
	return df.astype('string').drop_duplicates(UNIQUE_COLS, ignore_index=True)